# Implement Bert

Author: Crystal

Currently, it takes 9.663 seconds to run 100 abstracts. We are working with 690,815 abstract, which will take about 18.6 hours to run. 

In [1]:
#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd

import re

import nltk
#nltk.download() #input: punkt

from nltk import tokenize


import statistics #calculate mean and others

In [2]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
abstracts = pd.read_pickle("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/FR-cleaned-2021FEB24.pkl")

In [4]:
abstracts

,index,original index,PROJECT_ID,ABSTRACT,FY,ORG_COUNT,PI_COUNT,working_abstract,Start_Char,nchar,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,1,1,The multiprotein complex y-secretase proteolyt...,T,1402,...,BOSTON,MA,021156110,UNITED STATES,12/1/2007,1/1/2008,93.866,2008,3483.0,NaN
1,1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,1,1,The Kissl gene encodes peptides called kisspep...,T,2553,...,SEATTLE,WA,981959472,UNITED STATES,9/1/2008,1/1/2009,93.865,2008,39175.0,NaN
2,2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,1,1,The objective of this research is to understan...,T,1414,...,CAMBRIDGE,MA,021385319,UNITED STATES,1/2/2008,1/1/2009,93.859,2008,49646.0,NaN
3,3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,1,1,Obesity is the cause of many adverse pregnancy...,O,1545,...,HOUSTON,TX,770305400,UNITED STATES,4/1/2008,1/1/2009,93.361,2008,20406.0,NaN
4,4,371628,594482,Local potato advisory groups have expressed in...,2010,1,1,Local potato advisory groups have expressed in...,L,271,...,CORVALLIS,OR,97331,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690809,690850,1034758,1126666,Project Summary/AbstractThe pathophysiology of...,2019,1,1,The pathophysiology of schizophrenia is comple...,T,2836,...,RICHMOND,VA,232980568,UNITED STATES,9/10/2019,9/9/2020,93.242,2019,37602.0,NaN
690810,690851,1074034,1109788,PROJECT SUMMARY:Alzheimer’s Disease (AD) and A...,2019,1,1,Alzheimer’s Disease (AD) and Amyotrophic Later...,A,2899,...,WORCESTER,MA,016550002,UNITED STATES,9/10/2019,9/9/2020,93.866,2019,32209.0,NaN
690811,690852,1029450,1038442,"Project SummaryThe highest rates of mortality,...",2018,1,1,"The highest rates of mortality, acute care enc...",T,2186,...,NEW BRUNSWICK,NJ,089011914,UNITED STATES,9/10/2018,9/9/2019,93.840,2018,34994.0,NaN
690812,690853,1074409,1154490,PROJECT SUMMARY: Treatment paradigms for kidne...,2019,1,1,Treatment paradigms for kidney disease have be...,T,3056,...,CHICAGO,IL,606113152,UNITED STATES,9/10/2019,9/9/2020,93.847,2019,50016.0,NaN


# Functions

In [5]:
def get_corpus_embeddings(dir):
    with open(dir) as f:
        ai_text = f.read()
    ai_corpus = tokenize.sent_tokenize(ai_text) #sentence tokenization
    ai_embeddings = embedder.encode(ai_corpus, show_progress_bar=True) # embeddings
    return ai_embeddings


# k: number of similar sentences from AI corpus
# abstract: abstract from FEDERAL RePORTER
# print_result: if TRUE, print out the similar sentenses from AI corpus to each sentence in the abstract
def get_score(k, abstract, print_result = False):
    queries = tokenize.sent_tokenize(abstract) 

    # init a result list for scores
    result = []
    
    # Find the closest k sentences of the AI corpus for each query sentence (ML) based on cosine similarity
    top_k = min(k, len(ai_embeddings))
    
    for query in queries: #compare each sentence in the abstract to the ai corpus
        query_embedding = embedder.encode(query, show_progress_bar=False) 
        
        # We use cosine-similarity and torch.topk to find the highest k scores
        cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
        
        top_results = torch.topk(cos_scores, k=top_k)   #get the top k scores
        result.append(top_results.values.tolist()) #unlist the top result list
        if print_result:
            print("\n\n======================\n\n")
            print("Query:", query)
            print("Results:", top_results)
            print("\nTop k=5 most similar sentences in corpus:")
            for score, idx in zip(top_results[0], top_results[1]):
                print(ai_corpus[idx], "(Score: {:.4f})".format(score))
    return result

In [6]:
ai_embeddings = get_corpus_embeddings("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/ai_wiki_text.txt")

abstracts = abstracts.assign(score= abstracts["ABSTRACT"].apply(lambda x: get_score(10,x,False)))

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-c00f3c5256d0>", line 3, in <module>
    abstracts = abstracts.assign(score= abstracts["ABSTRACT"].apply(lambda x: get_score(10,x,False)))
  File "/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/pandas/core/series.py", line 4138, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/lib.pyx", line 2467, in pandas._libs.lib.map_infer
  File "<ipython-input-6-c00f3c5256d0>", line 3, in <lambda>
    abstracts = abstracts.assign(score= abstracts["ABSTRACT"].apply(lambda x: get_score(10,x,False)))
  File "<ipython-input-5-7061615d674e>", line 22, in get_score
    query_embedding = embedder.encode(query, show_progress_bar=False)
  File "/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-package

TypeError: object of type 'NoneType' has no len()

In [ ]:
sentence_score= []
for abstract in abstracts["score"]:
    sentence_score.append([statistics.mean(i) for i in abstract])

abstracts["sentence_score"]=sentence_score
abstracts

In [ ]:
abstracts.to_csv(r'/home/zz3hs/git/dspg21RnD/data/dspg21RnD/abstracts_embedding_score.csv', index = False)   